## 2. Keyword/Text search with Korean Analyzer

### Analyzer

- https://learn.microsoft.com/en-us/azure/search/search-analyzers
- https://learn.microsoft.com/en-us/azure/search/index-add-language-analyzers

In [2]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [3]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
)

In [4]:
import os
service_endpoint = os.getenv("AZSCH_ENDPOINT")  
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

print(service_endpoint)

https://iksch.search.windows.net


In [5]:
# Create a search index
def create_search_index(index_name):
    index_client = SearchIndexClient(
        endpoint=service_endpoint, credential=credential)
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="title", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="body", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="date", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True,
                        facetable=False, filterable=True),
        SearchableField(name="type", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True,
                        facetable=True, filterable=True),
        SearchableField(name="docid", type=SearchFieldDataType.String)
    ]

    # Create the search index
    index = SearchIndex(name=index_name, fields=fields)
    result = index_client.create_or_update_index(index)
    print(f' {result.name} created')

In [6]:
index_name = "ncevent-kor-index"
create_search_index(index_name)

 ncevent-kor-index created


In [7]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

In [8]:
import pandas as pd
from tqdm import tqdm

In [9]:
df = pd.read_json("./sample_events.json")
df.head()

,id,title,date,body
0,520783,"설날 기념 마이타임, 핫타임 이벤트",2025-01-24 12:00:00,\n자유를 향한 검은 날개\n안녕하세요. 나이트 크로우입니다.\n설날 기념 마이타임...
1,520257,설 기념 깜짝 우편 이벤트,2025-01-22 22:10:00,\n자유를 향한 검은 날개\n안녕하세요. 나이트 크로우입니다.\n2025년 설 연휴...
2,520255,"도노반, 에스텔라 전용 이벤트",2025-01-22 22:10:00,\n자유를 향한 검은 날개\n안녕하세요. 나이트 크로우입니다.\n독립된 또 다른 세...
3,520254,특별한 새해 떡국 이벤트,2025-01-22 22:10:00,\n자유를 향한 검은 날개\n안녕하세요. 나이트 크로우입니다.\n두 번째 설을 기념...
4,520253,+9 밤까마귀 장비 제작 이벤트,2025-01-22 22:10:00,\n자유를 향한 검은 날개\n안녕하세요. 나이트 크로우입니다.\n+9 밤까마귀 장비...


In [11]:
count = 0
batch_size = 40
for i in tqdm(range(0, len(df), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(df))
    
    documents = df[i:i_end].apply(
        lambda row: {'id': str(row.name), 
                     'title': row['title'], 
                     'body': row['body'],
                     'date': row['date'],
                     'type': "event",
                     "docid": str(row["id"])
                    }, axis=1).to_list()
    
    result = search_client.upload_documents(documents)  

100%|██████████| 7/7 [00:02<00:00,  2.96it/s]


In [12]:
from azure.search.documents.models import (
    QueryType,
    SearchMode
)

import json

def azsch_text_query(query, type=QueryType.Simple, mode=SearchMode.Any):

    results = search_client.search(  
        search_text=query,
        query_type=type,
        search_mode=mode,
        search_fields=["body", "title"],
        select=["docid", "body", "title", "date", "type"],
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 
    
    print("Search Results:")
    for i, result in enumerate(results, 1): 
        print(f"{i}) {result['@search.score']:.10f}: {result['docid']}, {result['title']}, {result['date']}")  


## Demo

- basic keyword search
- filter
- paging
- faceting

### basic search

In [13]:
results = azsch_text_query('산코나')

Search Results:
1) 11.1939520000: 264647, 출석 이벤트, 2024-03-27T18:30:00Z
2) 11.1015680000: 214626, 던전 이벤트, 2023-12-14T03:30:00Z
3) 10.7214160000: 474792, 던전 이벤트, 2024-11-07T09:52:00Z
4) 10.5787880000: 439637, 던전 이벤트, 2024-09-11T19:30:00Z
5) 10.2701640000: 344072, 출석 이벤트, 2024-05-22T18:30:00Z
6) 10.1351960000: 336151, 던전 이벤트, 2024-05-15T18:00:00Z
7) 9.5821840000: 224612, 던전 이벤트, 2024-01-24T18:30:00Z
8) 8.4938910000: 406041, 던전 이벤트, 2024-08-07T18:30:00Z
9) 8.0158500000: 336152, 부트캠프 전용 이벤트, 2024-05-15T18:00:00Z
10) 7.5101633000: 234729, 던전 이벤트, 2024-03-06T18:00:00Z


In [14]:
results = azsch_text_query('산코나유적')

Search Results:
1) 10.5293330000: 214626, 던전 이벤트, 2023-12-14T03:30:00Z
2) 10.1513190000: 474792, 던전 이벤트, 2024-11-07T09:52:00Z
3) 9.9466170000: 439637, 던전 이벤트, 2024-09-11T19:30:00Z
4) 9.8102650000: 264647, 출석 이벤트, 2024-03-27T18:30:00Z
5) 9.6310510000: 336151, 던전 이벤트, 2024-05-15T18:00:00Z
6) 9.2064100000: 344072, 출석 이벤트, 2024-05-22T18:30:00Z
7) 9.1241000000: 224612, 던전 이벤트, 2024-01-24T18:30:00Z
8) 8.0789150000: 406041, 던전 이벤트, 2024-08-07T18:30:00Z
9) 7.2428640000: 234729, 던전 이벤트, 2024-03-06T18:00:00Z
10) 7.0986010000: 376712, 던전 이벤트, 2024-06-26T18:30:00Z


### Hightlight

In [19]:
def azsch_text_query_highlight(query):

    results = search_client.search(  
        search_text=query,
        query_type=QueryType.Simple,
        search_mode=SearchMode.Any,
        search_fields=["body", "title"],
        select=["docid", "body", "title", "date", "type"],
        highlight_fields="body",
        query_language="ko-kr",
        top=5 # for limiting text search
    ) 
    
    print("Search Results:")
    for i, result in enumerate(results, 1):  
        if (result["@search.highlights"]):
            print(f"{result['docid']}: {result['@search.score']:.10f}: {result['title']}")
            for highlight in result['@search.highlights']['body']:
                print(f"    {highlight}")
        else:
            print(f"{result['docid']}: {result['@search.score']:.10f}: {result['title']} - {result['body']}")  

In [20]:
# facets
azsch_text_query_highlight("산코나유적")

Search Results:
214626: 10.5293330000: 던전 이벤트
    이벤트 기간 동안, <em>산코나</em> <em>유적에서</em> 더 많은 경험치와 무기 숙련도를 얻을 수 있게 되었습니다.
    지금 이 순간, <em>산코나</em> <em>유적에서</em> 누구보다 값진 성장을 향해 여러분의 여정을 시작하세요. 
    ■ 던전 이벤트
• 이벤트 기간: 12월 14일(목) 업데이트 후 ~ 12월 28일(목) 점검 전까지• 이벤트 내용: 이벤트 기간 동안 <em>산코나</em> <em>유적의</em> 경험치 획득량, 무기 숙련도 획득량이 증가합니다.이벤트 상세 내용이벤트 대상 지역혜택<em>산코나</em> <em>유적</em>경험치 +50%, 무기 숙련도 +100%

감사합니다.
474792: 10.1513190000: 던전 이벤트
    이벤트 기간 동안 <em>산코나</em> <em>유적에서</em> 더 많은 경험치와 무기 숙련도를 얻을 수 있게 되었습니다.
    이벤트 기간: 11월 7일(목) 업데이트 후 ~ 11월 21일(목) 업데이트 전까지이벤트 대상: 전체 서버이벤트 내용: 이벤트 기간 동안 <em>산코나</em> <em>유적의</em> 경험치 획득량, 무기 숙련도 획득량이 증가합니다.이벤트 상세 내용이벤트 대상 지역혜택<em>산코나</em> <em>유적</em>경험치 +80%, 무기 숙련도 +100%

감사합니다.
439637: 9.9466170000: 던전 이벤트
    <em>산코나</em> <em>유적과</em> 수련의 숲에서 더 높은 성과를 거두어 보시길 바랍니다.
    이벤트 기간: 9월 12일(목) 업데이트 후 ~ 9월 26일(목) 업데이트 전까지이벤트 내용: 이벤트 기간 동안 <em>산코나</em> <em>유적의</em> 경험치 획득량, 무기 숙련도 획득량이 증가합니다.이벤트 상세 내용이벤트 대상 지역혜택<em>산코나</em> <em>유적</em>경험치 +80%, 무기 숙련도 +100%

이벤트 기간: 